# YoloV5 Test
## Generate Image Dataset
### Imports

In [2]:
import cv2
import torch
import os
import numpy as np
from modules import image_processing as ip
import json

### Variable Settings

In [2]:
img_size= 240

#img_folder_path = 'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\YoloObjektDetection\\Images\\Dataset\\Srews_Nuts_Washers_Overlaps'
img_folder_path = 'E:\\Studierendenprojekte\\proj-camera-controller_\\Testing\\YoloObjektDetection\\Images\\Dataset\\Srews_Nuts_Washers_Overlaps'
image_name = '1.jpg'
image_path = img_folder_path + '\\' + image_name

#model_weights_path = 'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\yolov5_temp\\runs\\train\\Big\\weights\\best.pt'
model_weights_path = 'E:\\Studierendenprojekte\\proj-camera-controller_\\Testing\\yolov5_temp\\runs\\train\\Big\\weights\\best.pt'

labels = ['OverlapingObjects', 'Nut', 'Screw', 'Washer']

### Load YoloV5-Model pretrained with own data

In [3]:
def load_model(path):
    # Lade das trainierte Modell
    print(path)
    #model = torch.load(model_weights_path, map_location='cpu')
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=path)
    return model


model = load_model(path=model_weights_path)

E:\Studierendenprojekte\proj-camera-controller_\Testing\yolov5_temp\runs\train\Big\weights\best.pt


Using cache found in C:\Users\AI-Student/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-11-17 Python-3.9.12 torch-2.1.2+cpu CPU

Fusing layers... 
Model summary: 322 layers, 86193601 parameters, 0 gradients, 203.8 GFLOPs
Adding AutoShape... 


### Load Images

In [5]:
image_path='E:\\Studierendenprojekte\\proj-camera-controller_\\Testing\\YoloObjektDetection\\Images\\Dataset\\Srews_Nuts_Washers_Overlaps\\1.jpg'
img_folder_path = 'YoloObjektDetection/Images/dataset/Srews_Nuts_Washers_Overlaps/1.jpg'

image = cv2.imread(image_path)

# Überprüfen, ob das Bild erfolgreich geladen wurde
if image is not None:
    cv2.imshow('Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("[WARNING] No image loaded")
    

### Load more files of one Folder

In [5]:
img_folder_path = 'YoloObjektDetection/Images/dataset/Dataset_New'
img_array = []
# Überprüfen, ob der Ordner existiert
if os.path.isdir(img_folder_path):
    # Durchlaufe alle Dateien im Ordner
    for filename in os.listdir(img_folder_path):
        # Pfade für jedes Object erstellen
        image_path = os.path.join(img_folder_path, filename)

        # Überprüfen, ob es sich um eine Datei handelt
        if os.path.isfile(image_path):
            # Load Image
            image = cv2.imread(image_path)

            # Überprüfen, ob das Bild erfolgreich geladen wurde
            if image is not None:
                img_array.append(image)
            else:
                print(f"Unable to load {filename}")
        else:
            print(f"No File: {filename} ")
else:
    print("Path don't exists")

print(f'Loaded {len(img_array)} Images')


Loaded 24 Images


In [10]:
def get_image(imagenr):
    return img_array[imagenr][0]

def get_image_name(imagenr):
    return img_array[imagenr][1]

### Predict Results

In [7]:
def expand_rotated_box(box_point, offset):
    # Finde den Mittelpunkt der Boundingbox
    center = np.mean(box_points, axis=0)
    
    # Berechne die Differenzen der Eckpunkte zum Mittelpunkt
    diff = box_points - center
    
    # Berechne die Winkel der Eckpunkte zur x-Achse
    angles = np.arctan2(diff[:, 1], diff[:, 0])
    
    # Füge den Offset-Wert zu den Radien der Eckpunkte hinzu
    radii = np.sqrt(np.sum(diff ** 2, axis=1)) + offset
    
    # Berechne die neuen Koordinaten der erweiterten Boundingbox
    new_x = center[0] + radii * np.cos(angles)
    new_y = center[1] + radii * np.sin(angles)
    expanded_box = np.column_stack([new_x, new_y]).astype(int)
    return expanded_box

In [8]:
length_array = []
names_array = []
all_objects_array = []
all_standardized_array = []
object_images = []


for img_nr, img in enumerate(img_array):
    image = img
    image = image[175:950, 0:1700]
    output = image.copy()
    results = model(image)
    con_array = []
    
    cropped_boxes = []  # Liste für ausgeschnittene Boxen
    
    for box in results.xyxy[0].cpu().numpy():
        box = [float(i) for i in box]
        xmin, ymin, xmax, ymax, conf, cls = box
        
        # Cut the ROI
        region = image[int(ymin):int(ymax), int(xmin):int(xmax)]
        
        # Convert the ROI into greyscale
        cvt = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
        
        # Blur the ROI
        blur = cv2.medianBlur(cvt, 9)
        # Define threshold by OTSU Method      
        otsu_threshold, binary_image = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Anpassen des Schwellenwerts (z. B. um 10 Einheiten erhöhen)
        
        adjusted_threshold = otsu_threshold + 10
        
        # Anwenden des angepassten Schwellenwerts
        _, otsu_thresh = cv2.threshold(blur, adjusted_threshold, 255, cv2.THRESH_BINARY)

        # Detect the contours in the yoloV5-area box
        contours, _ = cv2.findContours(otsu_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Sort the detected contours by size of the area
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        
        # Use the biggest contur
        if contours:
            contour = contours[0]
        
            adjusted_contour = contour.copy()
            for i, point in enumerate(adjusted_contour):
                adjusted_contour[i][0][0] += xmin  # Verschiebe x-Wert um xmin
                adjusted_contour[i][0][1] += ymin  # Verschiebe y-Wert um ymin
            
            con_array.append(adjusted_contour)
            
            # Calculate rotated Rectangle
            rect = cv2.minAreaRect(adjusted_contour)
            box_points = cv2.boxPoints(rect)   
            box_points = np.int0(box_points)
            
            # Der Offset-Wert für die Erweiterung in alle Richtungen (Beispielwert)
            expanded_box = expand_rotated_box(box_points, 10)
            
            # Draw Red Box arround eatch item
            cv2.drawContours(output, [box_points], 0, (0, 0, 255), 2)
            cv2.drawContours(output, [expanded_box], 0, (255, 0, 0), 2)
            
        output = cv2.rectangle(output, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
        output = cv2.putText(output, labels[int(cls)], (int(xmin), int(ymin) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                             (0, 255, 0), 1, cv2.LINE_AA)    
    cv2.imshow("Output", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    rectangles = ip.get_rects_from_contours(con_array)
    bounding_boxes = ip.get_bounding_boxes_from_rectangles(rectangles)
    
    object_images = ip.warp_objects_horizontal(image, rectangles, bounding_boxes)
    for obj in object_images:
        all_objects_array.append(obj)
    
standardized = ip.standardize_images(all_objects_array)
for standard in standardized:
    all_standardized_array.append(standard)

for i, image in enumerate(all_standardized_array):
    path = 'Scanned Data/Part{}.png'.format(i)
    names_array.append('Part{}.png'.format(i))
    cv2.imwrite(path, image)


for i,_ in enumerate(all_objects_array):
    length_array.append(max(all_objects_array[i][0].shape))

data = {}
for idx, name in enumerate(names_array):
    data[name] = length_array[idx]

# Schreiben der Daten in eine JSON-Datei
with open('Dataset.json', 'w') as file:
    json.dump(data, file)
        



### Load Data from path by Json

In [141]:
import os
import cv2
path_Dataset = 'Scanned Data'
path_Json = 'Dataset.json'

# List of all Data in Dataset
file_names = os.listdir(path_Dataset)

# Load Data from Path
with open(path_Json, 'r') as file:
    loaded_data = json.load(file)

# Extract all Names and Length of Images
image_names_array = list(loaded_data.keys())  
length_array = list(loaded_data.values())    

# Load Images with CV2
images = []
for i in range(len(image_names_array)):
    image = cv2.imread(path_Dataset + '\\' + image_names_array[i])  

    if image is not None:
        images.append(image) 

# Output of Data
print("Names of Images: ", image_names_array)
print("Length: ", length_array)
print("Image Count: ", len(images))


Names of Images:  ['Part0.png', 'Part1.png', 'Part2.png', 'Part3.png', 'Part4.png', 'Part5.png', 'Part6.png', 'Part7.png', 'Part8.png', 'Part9.png', 'Part10.png', 'Part11.png', 'Part12.png', 'Part13.png', 'Part14.png', 'Part15.png', 'Part16.png', 'Part17.png', 'Part18.png', 'Part19.png', 'Part20.png', 'Part21.png', 'Part22.png', 'Part23.png', 'Part24.png', 'Part25.png']
Length:  [231, 156, 196, 78, 132, 158, 386, 160, 186, 229, 170, 85, 152, 98, 190, 154, 160, 94, 364, 175, 221, 36, 42, 378, 29, 29]
Image Count:  26



# Extract Conventional

In [6]:
DataFolderPath = 'Testing/Scanned Data'

In [7]:
image = cv2.imread(image_path)
output = image.copy()
results = model(image)

In [8]:
preprocessed_image = ip.image_preprocessing(image)
cv2.imshow('Test',preprocessed_image)
contours, rectangles, bounding_boxes, object_images = ip.get_objects_in_preprocessed_image(preprocessed_image,
                                                                                             smaller_image_area=True)
black_BG = ip.standardize_images(object_images,224)

#cv2.imshow('test',black_BG[0])

canvas_image = cv2.drawContours(preprocessed_image, bounding_boxes, -1, (0, 0, 255), 2)
cv2.imshow('canvas_image',canvas_image)

cv2.waitKey(0)
cv2.destroyAllWindows() 


    

# Backup

In [ ]:
image = cv2.imread(image_path)
image = image[175:950, 0:1700]
output = image.copy()
results = model(image)

cropped_boxes = []  # Liste für ausgeschnittene Boxen
con_array = []
lengh_array = []
names_array=[]

for box in results.xyxy[0].cpu().numpy():
    box = [float(i) for i in box]
    xmin, ymin, xmax, ymax, conf, cls = box
    
    # Cut the ROI
    region = image[int(ymin):int(ymax), int(xmin):int(xmax)]
    
    # Convert the ROI into greyscale
    cvt = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
    
    # Blur the ROI
    blur = cv2.medianBlur(cvt, 9)
    
    # Define threshold by OTSU Method      
    otsu_threshold, binary_image = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Anpassen des Schwellenwerts (z. B. um 10 Einheiten erhöhen)
    adjusted_threshold = otsu_threshold + 30

    # Anwenden des angepassten Schwellenwerts
    _, otsu_thresh = cv2.threshold(blur, adjusted_threshold, 255, cv2.THRESH_BINARY)
    
    # Detect the contours in the yoloV5-area box
    contours, _ = cv2.findContours(otsu_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Sort the detected contours by size of the area
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    # Use the biggest contur
    if contours:
        contour = contours[0]
        #print(f'contour {contour}')
    
        adjusted_contour = contour.copy()
        for i, point in enumerate(adjusted_contour):
            adjusted_contour[i][0][0] += xmin  # Verschiebe x-Wert um xmin
            adjusted_contour[i][0][1] += ymin  # Verschiebe y-Wert um ymin
        
        con_array.append(adjusted_contour)
        
        # Calculate rotated Rectangle
        rect = cv2.minAreaRect(adjusted_contour)
        box_points = cv2.boxPoints(rect)   
        box_points = np.int0(box_points)
        
        
        # Der Offset-Wert für die Erweiterung in alle Richtungen (Beispielwert)
        expanded_box = expand_rotated_box(box_points, 10)
        
        #print(f'boxpoibnts: {box_points.dtype, box_points.shape, box_points.size, box_points.ctypes}')
        #print(f'expanded_box: {expanded_box.dtype, expanded_box.shape, expanded_box.size, expanded_box.ctypes}')
        #box_points = expanded_box
        
        # Draw Red Box arround eatch item
        cv2.drawContours(output, [box_points], 0, (0, 0, 255), 2)
        cv2.drawContours(output, [expanded_box], 0, (255, 0, 0), 2)
        #cv2.imshow("Predicted Objects", output)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        
    output = cv2.rectangle(output, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 2)
    output = cv2.putText(output, labels[int(cls)], (int(xmin), int(ymin) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                         (0, 255, 0), 1, cv2.LINE_AA)    
    
rectangles = ip.get_rects_from_contours(con_array)

bounding_boxes = ip.get_bounding_boxes_from_rectangles(rectangles)

object_images = ip.warp_objects_horizontal(image, rectangles, bounding_boxes)
    
for i,_ in enumerate(object_images):
    lengh_array.append(max(object_images[i].shape))

'''
for i,image in enumerate(object_images):
    cv2.imshow('Single Object',object_images[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()
'''

standardized = ip.standardize_images(object_images)

'''
for i,image in enumerate(standardized):
    cv2.imshow('Single Object',img_array[i])
    cv2.waitKey(0)
    cv2.destroyAllWindows()
'''

for i, image in enumerate(standardized):
    path = 'E:\\Studierendenprojekte\\proj-camera-controller_\\Testing\\Scanned Data\\Part{}.png'.format(i)
    names_array.append('Part{}.png'.format(i))
    #path = 'C:\\Users\\doett\\Documents\\Workspace\\GitHub\\proj-sorting-robot\\Testing\\Scanned Data\\Item{}.png'.format(i)
    cv2.imwrite(path, image)
    #print(f'Image {i} saved')
    
data = {}
for idx, name in enumerate(names_array):
    data[name] = lengh_array[idx]
    
with open('bildlaengen.json', 'w') as file:
    json.dump(data, file)

# Schreiben der Daten in eine JSON-Datei
with open('E:\\Studierendenprojekte\\proj-camera-controller_\\Testing\\Dataset.json', 'w') as file:
    json.dump(data, file)
    